## **Técnicas de Imputación Datos NASA**

#### **Imputación usando promedio estacional NASA**

In [ ]:
df_comparado['ALLSKY_SFC_SW_DWN_IMPUTACION'] = df_comparado['ALLSKY_SFC_SW_DWN'].copy()
df_comparado['ALLSKY_SFC_SW_DWN_IMPUTACION'] = df_comparado['ALLSKY_SFC_SW_DWN_IMPUTACION'].replace(-999.0, pd.NA)
df_comparado['datetime'] = pd.to_datetime(df_comparado['datetime'])
df_comparado['dayofyear'] = df_comparado['datetime'].dt.dayofyear
df_comparado['hour'] = df_comparado['datetime'].dt.hour
tabla_promedios_nasa = df_comparado.groupby(['dayofyear', 'hour'])['ALLSKY_SFC_SW_DWN_IMPUTACION'].mean()
def imputar_estacional_nasa(row):
    if pd.isna(row['ALLSKY_SFC_SW_DWN_IMPUTACION']):
        return tabla_promedios_nasa.get((int(row['dayofyear']), int(row['hour'])), 0)
    else:
        return row['ALLSKY_SFC_SW_DWN_IMPUTACION']
df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'] = df_comparado.apply(imputar_estacional_nasa, axis=1)
df_comparado.tail(10)

#### **Evolución de Radiación Solar Original vs Imputada (Promedio Estacional-NASA)**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'],
    mode='lines',
    name='INMET-IMPUTADA',
    line=dict(color='blue'),
    opacity=0.7
))
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['ALLSKY_SFC_SW_DWN'],
    mode='lines',
    name='INMET-SIN IMPUTAR',
    line=dict(color='orange'),
    opacity=0.7
))
fig.update_layout(
    title='Evolución de Radiación NASA - (Imputación Promedio Estacional) (2019-2025)',
    xaxis_title='Fecha y Hora',
    yaxis_title='Radiación (KJ/m²)',
    legend_title='Fuente',
    template='plotly_white',
    height=500,
    width=1100
)
fig.show()

#### **Distribución de Radiación Solar Original vs Imputada (Promedio Estacional-NASA)**

In [ ]:
rad_orig = df_comparado['ALLSKY_SFC_SW_DWN'][df_comparado['ALLSKY_SFC_SW_DWN'] > 0]
rad_imp = df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'][df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'] > 0]

fig, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)

sns.histplot(rad_orig, bins=60, kde=True, color='royalblue', ax=axes[0])
axes[0].set_title('ANTES')
axes[0].set_xlabel('Radiación (KJ/m²)')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True)

sns.histplot(rad_imp, bins=60, kde=True, color='darkorange', ax=axes[1])
axes[1].set_title('DESPUES')
axes[1].set_xlabel('Radiación (KJ/m²)')
axes[1].set_ylabel('')
axes[1].grid(True)

fig.suptitle('Distribuciones de Radiación Solar INMET ((Imputación Promedio Estacional)) (> 0) - Antes y Después de Imputación', fontsize=16)
plt.tight_layout()
plt.show()

#### **Evaluación de Radiación Solar Original vs Imputada (Promedio Estacional-NASA)**

In [ ]:
rad_orig = df_comparado['ALLSKY_SFC_SW_DWN'][df_comparado['ALLSKY_SFC_SW_DWN'] > 0]
rad_imp = df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'][df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'] > 0]
stat_ks, p_ks = ks_2samp(rad_orig, rad_imp)
print("Kolmogorov-Smirnov Test (Distribución Original vs Imputada):")
print(f"Estadístico KS: {stat_ks:.4f}")
print(f"p-valor: {p_ks:.4e}")
if p_ks > 0.05:
    print("No se rechaza H0 → Las distribuciones son estadísticamente similares.")
else:
    print("Se rechaza H0 → Las distribuciones son diferentes.")

## **Visualización de Radiación Imputadas INMET-NASA**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['RADIACAO_GLOBAL_IMPUTADA'],
    mode='lines',
    name='INMET',
    line=dict(color='blue'),
    opacity=0.7
))
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'],
    mode='lines',
    name='NASA',
    line=dict(color='orange'),
    opacity=0.7
))
fig.update_layout(
    title='Evolución de Radiación Horaria - INMET vs. NASA (2019-2025)',
    xaxis_title='Fecha y Hora',
    yaxis_title='Radiación (KJ/m²)',
    legend_title='Fuente',
    template='plotly_white',
    height=500,
    width=1100
)
fig.show()

### **Evaluación de Radiación Solar Imputadas INMET vs NASA**

In [ ]:
rad_orig = df_comparado['RADIACAO_GLOBAL_IMPUTADA'][df_comparado['RADIACAO_GLOBAL_IMPUTADA'] > 0]
rad_imp = df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'][df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'] > 0]

fig, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)

sns.histplot(rad_orig, bins=60, kde=True, color='royalblue', ax=axes[0])
axes[0].set_title('INMET (> 0)')
axes[0].set_xlabel('Radiación (KJ/m²)')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True)

sns.histplot(rad_imp, bins=60, kde=True, color='darkorange', ax=axes[1])
axes[1].set_title('NASA (> 0)')
axes[1].set_xlabel('Radiación (KJ/m²)')
axes[1].set_ylabel('')
axes[1].grid(True)

fig.suptitle('Distribuciones de Radiación Solar INMET - NASA', fontsize=16)
plt.tight_layout()
plt.show()

### **Evaluación de Radiación Solar Imputadas INMET vs NASA**

In [ ]:
rad_orig = df_comparado['RADIACAO_GLOBAL_IMPUTADA'][df_comparado['RADIACAO_GLOBAL_IMPUTADA'] > 0]
rad_imp = df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'][df_comparado['ALLSKY_SFC_SW_DWN_ESTACIONAL'] > 0]
stat_ks, p_ks = ks_2samp(rad_orig, rad_imp)
print("Kolmogorov-Smirnov Test (Distribución Original vs Imputada):")
print(f"Estadístico KS: {stat_ks:.4f}")
print(f"p-valor: {p_ks:.4e}")
if p_ks > 0.05:
    print("No se rechaza H0 → Las distribuciones son estadísticamente similares.")
else:
    print("Se rechaza H0 → Las distribuciones son diferentes.")